#### IMPORTS

In [9]:
#Importa os módulos usados

import numpy as np
import matplotlib.pyplot as plt
import copy


# Define um tipo de dado similar ao Pascal "record" or C "struct"

class Struct:
    pass

#### Solução Inicial

In [10]:
# Solução inicial
def sol_inicial(probdata, apply_heur):
    
    if True: #apply_heur == False: # EDITAR DEPOIS DE IMPLEMENTAR METAHEURISTICA
        x = Struct()
        x.solution = np.random.randint(0,1,size=(probdata.nCL, probdata.nPA))
        y = Struct()
        y.solution = np.random.randint(0,1,size=(1, probdata.nPA))
    
    else:
        x = Struct()
        x.solution = np.zeros((probdata.nCL, probdata.nPA))
        y = Struct()
        y.solution = np.zeros((probdata.nPA))

        # IMPLEMENTAR METAHEURÍSTICA CONSTRUTIVA AQUI
    
    return x, y

#### Funções UTIL para definição de dados

In [11]:
class Cliente:
    def __init__(self, x, y, consumo_banda):
        self.x = x
        self.y = y
        self.consumo_banda = consumo_banda

    def __repr__(self):
        return f"Cliente(x={self.x}, y={self.y}, consumo_banda={self.consumo_banda})"

# Função para ler os clientes do arquivo
def ler_clientes(nome_arquivo):
    clientes = []
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        for linha in linhas:
            x, y, consumo_banda = map(float, linha.strip().split(','))
            cliente = Cliente(x, y, consumo_banda)
            clientes.append(cliente)
    return clientes

# Uso da função para ler os clientes do arquivo
clientes = ler_clientes('data/clientes.csv')

# Exemplo de uso: imprimir os clientes lidos
for cliente in clientes:
    print(cliente)

Cliente(x=36.524, y=141.41, consumo_banda=1.4577)
Cliente(x=68.055, y=53.432, consumo_banda=0.18643)
Cliente(x=1.2006, y=65.695, consumo_banda=1.12)
Cliente(x=171.29, y=68.62, consumo_banda=0.96504)
Cliente(x=90.213, y=113.73, consumo_banda=0.89939)
Cliente(x=21.83, y=131.84, consumo_banda=0.75796)
Cliente(x=86.03, y=72.527, consumo_banda=0.86042)
Cliente(x=7.8241, y=64.146, consumo_banda=0.65123)
Cliente(x=109.12, y=126.68, consumo_banda=0.13283)
Cliente(x=70.927, y=116.16, consumo_banda=0.45007)
Cliente(x=82.1, y=104.41, consumo_banda=0.85078)
Cliente(x=98.092, y=243.31, consumo_banda=0.95733)
Cliente(x=122.69, y=81.585, consumo_banda=1.3975)
Cliente(x=39.511, y=97.064, consumo_banda=1.4671)
Cliente(x=6.6475, y=75.823, consumo_banda=0.15852)
Cliente(x=74.805, y=110.82, consumo_banda=0.99937)
Cliente(x=38.854, y=53.75, consumo_banda=0.9121)
Cliente(x=101.25, y=130.39, consumo_banda=0.72125)
Cliente(x=25.468, y=171.46, consumo_banda=0.54726)
Cliente(x=122.93, y=84.264, consumo_banda=0.

In [17]:
def clientProcessing(clientList, nCL=495, nPA=81**2):

    cons = np.zeros(nCL)
    dist = np.zeros((nCL, nPA))
    exp = np.zeros((nCL, nPA))

    cl_index = 0
    for cliente in clientList:
        cons[cl_index] = cliente.consumo_banda

        for PA_index in range(nPA):
            PA_x = (PA_index % 81) * 5
            PA_y = (PA_index // 81)
            dist[cl_index, PA_index] = np.sqrt((cliente.x - PA_x)**2 + (cliente.y - PA_y)**2)
            exp[cl_index, PA_index] = 1/dist[cl_index, PA_index]

        cl_index += 1

    return cons, dist, exp

C, D, E = clientProcessing(clientes)


68.055-53.432 to 0-0
[[146.05064422 144.88116053 143.87602537 ... 358.77077734 363.69799378
  368.62717029]
 [ 86.52434137  82.64932939  78.90096101 ... 323.03938096 328.0226999
  333.00651893]
 [ 65.70596978  65.8047754   66.28169027 ... 389.06247116 394.05913321
  399.05587888]
 ...
 [349.01834171 348.27106232 347.59410646 ... 427.2764244  431.21232919
  435.17008497]
 [319.36755706 317.32560013 315.34970509 ... 332.63348674 336.51885014
  340.43330698]
 [358.83537186 356.56797963 354.35663406 ... 328.30020423 331.75129857
  335.2414415 ]]


#### Função definição de dados para o problema

In [ ]:
def probdef(nPA=81**2, nCL=495):
    
    probdata = Struct()

    probdata.CL_cons = 0 # importar consumo dos clientes do .csv # TODO

    probdata.PA_cap = 54 # em Mbps
    probdata.PA_raio = 84 # em metros
    probdata.CL_min_p = 0.05 # porcentagem minima de clientes
    probdata.exp_coef = 1 # coeficiente de exposição
    probdata.falloff = 1 # fator de decaimento

    # CALCULAR DEPOIS
    probdata.dist_CL_PA = np.zeros((nCL, nPA)) # distância entre clientes e PAs # TODO
    probdata.exp_CL_PA = np.zeros((nCL, nPA)) # exposição dos clientes aos PAs # TODO

    probdata.nPA = nPA
    probdata.nCL = nCL
    probdata.nPA_max = 30

    return probdata

#### Função penalidades

In [5]:
# Função de penalidades para todas as restrições
def penalties(x, y, probdata):

    # percentual mínimo de clientes
    pen_CLmin = probdata.nCL * probdata.CL_min_p - np.sum(x.solution) 
    pen_CLmin = np.sum(np.where(pen_CLmin <= 0, 0, pen_CLmin)**2)

    # limite de consumo dos PAs
    pen_PAcap = np.zeros(probdata.nPA)
    for i in range(probdata.nPA):
        pen_PAcap[i] = np.dot(np.array(probdata.CL_cons), np.array(x.solution[:[i]])) - y.solution[i] * probdata.PA_cap
        pen_PAcap[i] = np.sum(np.where(pen_PAcap[i] <= 0, 0, pen_PAcap[i])**2)
    
    # limite de distância entre PAs e clientes
    pen_dist = np.zeros((probdata.nCL, probdata.nPA))
    for i in range(probdata.nPA):
        for j in probdata.nCL:
            pen_dist[j,i] = probdata.dist_CL_PA[j,i] * x.solution[j,i] - y.solution[i] * probdata.PA_raio
            pen_dist[j,i] = np.sum(np.where(pen_dist[j,i] <= 0, 0, pen_dist[j,i])**2)

    # pelo menos 5% de exposição à rede
    pen_CLmin = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_CLmin[j] = 0.05 * 1 - np.dot(np.array(probdata.exp_CL_PA[j]), np.array(y.solution))
        pen_CLmin[j] = np.sum(np.where(pen_CLmin[j] <= 0, 0, pen_CLmin[j])**2)
    
    # no máximo um PA por cliente
    pen_PAperCL = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_PAperCL[j] = sum(x.solution[j]) - 1
        pen_PAperCL[j] = np.sum(np.where(pen_PAperCL[j] <= 0, 0, pen_PAperCL[j])**2)

    # número máximo de PAs
    pen_PAmax = sum(y.solution) - probdata.nPA_max

    # return all multiplied by U
    return 100 * (pen_CLmin + sum(pen_PAcap) + sum(pen_dist) + sum(pen_CLmin) + sum(pen_PAperCL) + pen_PAmax)

#### Funções Objetivo

In [6]:
# Função objetivo 1: Minimizar número de PAs ativos
def fobj_minPA(x, y, probdata):

    sol = np.transpose(np.array(y.solution))

    y.fitness = np.sum(sol)
    y.penalidade = penalties(x, y, probdata)
    y.fitness_penalizado = y.fitness + y.penalidade

    return y


# Função objetivo 2: Minimizar distância cumulativa de clientes e PAs
def fobj_mindist(x, y, probdata):

    sol = x.solution

    fit_matrix = np.multiply(sol, probdata.dist_CL_PA)
    x.fitness = sum(fit_matrix)
    x.penalidade = penalties(x, y, probdata)
    x.fitness_penalizado = x.fitness + x.penalidade

    return x

#### Funções UTIL para VNS

In [8]:
# NeighborhoodChange implementation
def neighborhoodChange(x, xlinha, k):
    
    if xlinha.fitness_penalizado < x.fitness_penalizado:
        x = copy.deepcopy(xlinha)
        k  = 1
    else:
        k += 1
        
    return x, k


# Shake implementation // TODO: MUDAR HEURISTICAS
def shake(x,k,probdata):
    
    y = copy.deepcopy(x)
    r = np.random.permutation(probdata.n)       
    
    if k == 1:             # apply not operator in one random position
        y.solution[r[0]] = not(y.solution[r[0]])
        
    elif k == 2:           # apply not operator in two random positions        
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        
    elif k == 3:           # apply not operator in three random positions
        y.solution[r[0]] = not(y.solution[r[0]])
        y.solution[r[1]] = not(y.solution[r[1]])
        y.solution[r[2]] = not(y.solution[r[2]])        
    
    return y

#### VNS IMPLEMENTATION